In [642]:
import pandas as pd
import numpy as np
import os
import csv
import os.path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso
from sklearn.svm import SVR

In [624]:
# working_dir = os.path.abspath(os.path.dirname('data_4_ScienceHack2021/'))

In [625]:
working_dir = '/Users/nikanizharadze/Desktop/data_4_ScienceHack2021/'

In [626]:
folder_name = 'trip_'
gps_filename = 'gps_data.csv'
passanger_count_filename = 'passenger_count_nums'
all_trips = []
# for i in range(1,13):
for i in range(1,13):
    df_gps = pd.read_csv(working_dir+folder_name+str(i) +'/' + gps_filename)
    df_passanger_count = pd.read_csv(working_dir+folder_name+str(i) +'/' + passanger_count_filename)
    # Remove Nan, inf values in passanger_count file
    df_passanger_count = df_passanger_count[~df_passanger_count.isin([np.nan, np.inf, -np.inf]).any(1)]
    # # Convert epoch_ts to int so GPS dataframe and Passanger_Count dataframe are mergable on epoch_ts
    df_passanger_count['Epoch_Time'] = df_passanger_count['Epoch_Time'].astype(float).round().astype(int)
    df_passanger_count = df_passanger_count.rename(columns={'Epoch_Time': 'epoch_ts'})
    merged = df_gps.merge(df_passanger_count, on="epoch_ts", how='left')
    merged = merged.fillna(0)
    door_closed_indeces = merged[merged.Number_of_Passengers==-10].index.values
    door_open_indeces = merged[merged.Number_of_Passengers==-99].index.values
    for j in range(len(door_closed_indeces)):
        value_to_insert = sum(merged.iloc[door_closed_indeces[j]+1:door_open_indeces[j]-1]['Number_of_Passengers'])
        merged.loc[door_closed_indeces[j]:door_open_indeces[j]]['Number_of_Passengers'] =  value_to_insert   
    all_trips.append(merged)

/Users/nikanizharadze/anaconda3/envs/PricePrediction/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [627]:
len(all_trips)

12

In [628]:
# all_trips contain 12 dataframes each of which merges relevant gps data to number of passengers data based on epoch_ts

In [629]:
# Removes is_valid == False
for i in range(len(all_trips)):
    all_trips[i] = all_trips[i][all_trips[i].is_valid]

In [630]:
# drop columns is_valid, lat_dir, lon_dir
for i in range(len(all_trips)):
    del all_trips[i]['is_valid']
    del all_trips[i]['lon_dir']
    del all_trips[i]['lat_dir']
    all_trips[i] = all_trips[i][all_trips[i].Number_of_Passengers != 0]

In [631]:
# append dataframes 
df = pd.DataFrame()
for i in range(len(all_trips)-1):
    df = df.append(all_trips[i+1], ignore_index=True)

In [632]:
df = df.sort_values(by='date_time',ascending=True)
df = df.reset_index(drop=True)


In [633]:
df.head()

,date_time,latitude,longitude,speed_over_ground_knots,heading_true_course,epoch_ts,Number_of_Passengers
0,2021-04-12 10:26:41,48.128255,11.604194,2.911,0.00,1.618223e+09,15.0
1,2021-04-12 10:26:42,48.128256,11.604222,3.345,0.00,1.618223e+09,15.0
2,2021-04-12 10:26:43,48.128276,11.604249,3.486,23.64,1.618223e+09,15.0
3,2021-04-12 10:26:44,48.128284,11.604271,3.324,47.95,1.618223e+09,15.0
4,2021-04-12 10:26:45,48.128289,11.604291,3.164,58.89,1.618223e+09,15.0


In [634]:
Y_cols = ['Number_of_Passengers'] 
X_cols = [col for col in list(df) if col not in Y_cols]

In [635]:
X_cols

['date_time',
 'latitude',
 'longitude',
 'speed_over_ground_knots',
 'heading_true_course',
 'epoch_ts']

In [636]:
# convert date to float for machine learning models
df['date_time'] = pd.to_datetime(df['date_time'])

In [637]:
df['date_time'] = (df['date_time'] - df['date_time'].min())  / np.timedelta64(1,'D')

In [638]:
df.head()

,date_time,latitude,longitude,speed_over_ground_knots,heading_true_course,epoch_ts,Number_of_Passengers
0,0.000000,48.128255,11.604194,2.911,0.00,1.618223e+09,15.0
1,0.000012,48.128256,11.604222,3.345,0.00,1.618223e+09,15.0
2,0.000023,48.128276,11.604249,3.486,23.64,1.618223e+09,15.0
3,0.000035,48.128284,11.604271,3.324,47.95,1.618223e+09,15.0
4,0.000046,48.128289,11.604291,3.164,58.89,1.618223e+09,15.0


In [639]:
Y = df[Y_cols]
X = df[X_cols]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, shuffle=False, random_state=42)

In [640]:
ss1 = MinMaxScaler(feature_range=(0, 1))
X_train = ss1.fit_transform(X_train)
X_test = ss1.transform(X_test)
ss2 = MinMaxScaler(feature_range=(0, 1))
Y_train = ss2.fit_transform(Y_train)
Y_test = ss2.transform(Y_test)


In [616]:
# Lasso 

lasso = Lasso(alpha=0.0001)
        
lasso.fit(X_train, Y_train) 

Lasso(alpha=0.0001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [617]:
scaled_predictions = lasso.predict(X_test)

In [618]:
predictions = ss2.inverse_transform(scaled_predictions.reshape(-1, 1))

In [619]:
predictions

array([[14.8867567 ],
       [14.91788598],
       [14.93251453],
       ...,
       [14.53478558],
       [14.47329864],
       [14.4839792 ]])

In [620]:
ground_truth = ss2.inverse_transform(Y_test)

In [621]:
def MAE(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Absolute Error """
    return mean_absolute_error(actual, predicted)

def MSE(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Squared Error """
    return mean_squared_error(actual, predicted)

def Evaluate(Y_true, predictions):
    print('MAE '+str(MAE(Y_true, predictions)))
    print('MSE '+str(MSE(Y_true, predictions)))

In [622]:
Evaluate(ground_truth,predictions)

MAE 6.989160203350813
MSE 66.53736020430901


In [525]:
# Lasso does not perform too well ))

In [521]:
# SVR (Support Vector Regression)

In [643]:
svr = SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.001, gamma=0.1,
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
svr.fit(X_train,Y_train.ravel())

SVR(C=1000.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.001, gamma=0.1,
    kernel='linear', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [649]:
scaled_predictions = svr.predict(X_test)

In [650]:
predictions = ss2.inverse_transform(scaled_predictions.reshape(-1, 1))

In [651]:
predictions

array([[12.82614743],
       [12.84507557],
       [12.85652966],
       ...,
       [12.89574505],
       [12.85650725],
       [12.86333258]])

In [652]:
ground_truth = ss2.inverse_transform(Y_test)

In [653]:
Evaluate(ground_truth,predictions)

MAE 5.694549465947447
MSE 45.6747039007915
